![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [100]:
import pandas as pd
import numpy as np

# Start coding here...

In [101]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


### SUBSETTING AND CLEANING THE CLIENT'S TABLE
1. We subset for the client's table, `client_df` , columns from the larger dataframe
2. We clean the data to the requirements

We can use the `loc` accessor to change the values of `'unknown'` in the `education` column of the clients dataframe to np.NAN and in the same way, we can use the formula displayed below

- **client_df['education'] = client_df['education'].mask(client_df['education'] == 'unknown', np.NAN)**
- 
            



In [102]:
# Subseting the client's table from the entire bank dataframe
client_df = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]
# Cleaning the client's dataframe
# Replacing '.' with '_'
client_df['job'] = client_df['job'].str.replace('\.', '_')
client_df[client_df['job'].str.contains('\_')]['job'].value_counts()
# Cleaning the education column
# Replacing '.' with '_'
client_df['education'] = client_df['education'].str.replace('\.', '_')
client_df[client_df['education'].str.contains('\_')]['education'].value_counts()
# Replacing 'unknown' with np.NAN 
client_df.loc[client_df['education'] == 'unknown', 'education'] = np.NAN
client_df['education'].str.contains('unknown').value_counts()
# Cleaning hte credit_default column
# Ckecking the type
client_df['credit_default'] = np.where(client_df['credit_default'] == 'yes', 1, 0)
# changing the data type to boolean
client_df['credit_default'] = client_df['credit_default'].astype(bool)
client_df['credit_default'].value_counts()
client_df['credit_default'].dtype
# Cleaning hte mortgage column
# Ckecking the type
client_df['mortgage'] = np.where(client_df['mortgage'] == 'yes', 1, 0)
client_df['mortgage'] = client_df['mortgage'].astype('bool')
client_df['mortgage'].dtype
client_df['mortgage'].value_counts()
print(client_df.info())
client_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB
None


,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False
...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,False,True
41184,41184,46,blue-collar,married,professional_course,False,False
41185,41185,56,retired,married,university_degree,False,True
41186,41186,44,technician,married,professional_course,False,False


### SUBSETTING AND CLEANING THE CAMPAIGN'S TABLE

1. We subset for the campaign's table, campaign_df , columns from the larger dataframe
2. We clean the data to the requirements

We can use the loc accessor to change the values of 'unknown' in the education column of the clients dataframe to np.NAN and in the same way, we can use the formula displayed below

In [103]:
# Subseting the client's table from the entire bank dataframe
campaign_df = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]
# Converting the 'previous_outcome' columns to a boolean data type
campaign_df['previous_outcome'] = np.where(campaign_df['previous_outcome'] == 'success', 1, 0).astype(bool)
# Converting the 'campaign_outcome' column to a boolean datatype
campaign_df['campaign_outcome'] = np.where(campaign_df['campaign_outcome'] == 'yes', 1, 0).astype(bool)
df['year'] = 2022
campaign_df.head()
campaign_df['last_contact_date'] = df['year'].astype(str) + '-' + df['month'].astype(str) + '-' + df['day'].astype(str)
campaign_df['last_contact_date'] = pd.to_datetime(campaign_df['last_contact_date'])
print(campaign_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   number_contacts             41188 non-null  int64         
 2   contact_duration            41188 non-null  int64         
 3   previous_campaign_contacts  41188 non-null  int64         
 4   previous_outcome            41188 non-null  bool          
 5   campaign_outcome            41188 non-null  bool          
 6   last_contact_date           41188 non-null  datetime64[ns]
dtypes: bool(2), datetime64[ns](1), int64(4)
memory usage: 1.6 MB
None


### SUBSETTING AND CLEANING THE ECONOMIC'S TABLE

1. We subset for the economics's table, economics_df , columns from the larger dataframe
2. We clean the data to the requirements

We can use the loc accessor to change the values of 'unknown' in the education column of the clients dataframe to np.NAN and in the same way, we can use the formula displayed below

In [104]:
# Subseting the client's table from the entire bank dataframe
economics_df = df[['client_id', 'cons_price_idx', 'euribor_three_months']]
print(economics_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   cons_price_idx        41188 non-null  float64
 2   euribor_three_months  41188 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 965.5 KB
None


### SAVING THE FILES

1. `client_df` as `client.csv`
2. `campaign_df` as `campaign.csv`
3. `economics_df` as `economics.csv`

In [105]:
# Saving all the dataframes to a CSV file 
client_df.to_csv('client.csv', index= False)
campaign_df.to_csv('campaign.csv', index= False)
economics_df.to_csv('economics.csv', index = False)